In [ ]:
%%javascript
  function ClickConnect() {
    var iconElement = document.getElementById("toggle-header-button");
    if (iconElement) {
    var clickEvent = new MouseEvent ("click", {
        bubbles: true, cancelable: true,
        view: window
      });
    iconElement.dispatchEvent (clickEvent);
    }
  }
  setInterval(ClickConnect, 6000);

<IPython.core.display.Javascript object>

In [ ]:
!pip install tflearn tensorflow
!pip install --upgrade tflearn

In [ ]:
!pip show tflearn
%cd /usr/local/lib/python3.10/dist-packages/tflearn/layers
!sed -i 's/from tensorflow.python.util.nest import is_sequence/from tensorflow.python.util.nest import is_sequence_or_composite as is_sequence/' recurrent.py
!grep 'is_sequence' recurrent.py

Name: tflearn
Version: 0.5.0
Summary: Deep Learning Library featuring a higher-level API for TensorFlow
Home-page: https://github.com/tflearn/tflearn
Author: TFLearn contributors
Author-email: aymeric.damien@gmail.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, Pillow, six
Required-by: 
/usr/local/lib/python3.10/dist-packages/tflearn/layers
from tensorflow.python.util.nest import is_sequence_or_composite as is_sequence_or_composite as is_sequence_or_composite as is_sequence_or_composite as is_sequence_or_composite as is_sequence_or_composite as is_sequence_or_composite as is_sequence
    if args is None or (is_sequence(args) and not args):
    if not is_sequence(args):


In [ ]:
%cd /usr/local/lib/python3.10/dist-packages/tflearn
!sed -i 's/Image.ANTIALIAS/Image.LANCZOS/' data_utils.py
!grep 'Image.LANCZOS' data_utils.py

/usr/local/lib/python3.10/dist-packages/tflearn
                 resize_mode=Image.LANCZOS):


In [ ]:
import os
import random
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tflearn
from tflearn.data_utils import to_categorical
import tensorflow
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def cargarDataset(ruta_dataset, tamanio_imagen=(300, 300), test_size=0.2):
    dataset = []
    etiquetas = []
    etiquetas_mapping = {}
    clases = sorted(os.listdir(ruta_dataset))
    etiquetas_mapping = {clase: idx for idx, clase in enumerate(clases)}
    for clase in clases:
        ruta_clase = os.path.join(ruta_dataset, clase)
        etiqueta_num = etiquetas_mapping[clase]
        with os.scandir(ruta_clase) as entries:
            for entry in entries:
                if entry.is_file():
                    imagen_ruta = os.path.join(ruta_clase, entry.name)
                    imagen = Image.open(imagen_ruta).convert('RGB')
                    imagen_resized = imagen.resize(tamanio_imagen)
                    imagen_array = np.array(imagen_resized) / 255.0
                    dataset.append(imagen_array)
                    etiquetas.append(etiqueta_num)
    dataset = np.array(dataset, dtype=np.float32)
    etiquetas = np.array(etiquetas, dtype=np.int32)
    X_train, X_test, y_train, y_test = train_test_split(dataset, etiquetas, test_size=test_size, random_state=42)
    return (X_train, y_train), (X_test, y_test)

In [ ]:
tensorflow.compat.v1.reset_default_graph()

In [ ]:
(X_train, y_train), (X_test, y_test) = cargarDataset('/content/drive/MyDrive/SIS421/Laboratorio N°1 - SIS421/Plantas')

In [ ]:
input_dim = 300 * 300 * 3
hidden_units = 256
output_classes = 5

input_layer = tflearn.input_data(shape=[None, input_dim])
hidden_layer = tflearn.fully_connected(input_layer, hidden_units, activation='relu')
output_layer = tflearn.fully_connected(hidden_layer, output_classes, activation='softmax')
network = tflearn.regression(output_layer, optimizer='sgd', learning_rate=0.001, loss='categorical_crossentropy')
model = tflearn.DNN(network)

In [ ]:
X_train = X_train.reshape(-1, input_dim)
X_test = X_test.reshape(-1, input_dim)

y_train = to_categorical(y_train, nb_classes=output_classes)
y_test = to_categorical(y_test, nb_classes=output_classes)

print(f"Forma de X_train después de la corrección: {X_train.shape}")
print(f"Forma de y_train después de la corrección: {y_train.shape}")

Forma de X_train después de la corrección: (8000, 270000)
Forma de y_train después de la corrección: (8000, 5)


In [ ]:
model.fit(X_train, y_train, n_epoch=1000, batch_size=1000, show_metric=True)
accuracy = model.evaluate(X_test, y_test)
print(f"Precisión con todo el dataset: {accuracy[0]:.4f}")

Streaming output truncated to the last 5000 lines.
--
Training Step: 8702  | total loss: 0.62993 | time: 0.570s
| SGD | epoch: 737 | loss: 0.62993 - acc: 0.8109 -- iter: 1000/8000
Training Step: 8703  | total loss: 0.62657 | time: 1.141s
| SGD | epoch: 737 | loss: 0.62657 - acc: 0.8130 -- iter: 2000/8000
Training Step: 8704  | total loss: 0.63055 | time: 1.700s
| SGD | epoch: 737 | loss: 0.63055 - acc: 0.8105 -- iter: 3000/8000
Training Step: 8705  | total loss: 0.62693 | time: 2.271s
| SGD | epoch: 737 | loss: 0.62693 - acc: 0.8116 -- iter: 4000/8000
Training Step: 8706  | total loss: 0.62300 | time: 2.841s
| SGD | epoch: 737 | loss: 0.62300 - acc: 0.8139 -- iter: 5000/8000
Training Step: 8707  | total loss: 0.62015 | time: 3.420s
| SGD | epoch: 737 | loss: 0.62015 - acc: 0.8166 -- iter: 6000/8000
Training Step: 8708  | total loss: 0.62270 | time: 3.993s
| SGD | epoch: 737 | loss: 0.62270 - acc: 0.8147 -- iter: 7000/8000
Training Step: 8709  | total loss: 0.62403 | time: 4.561s
| SGD 